# Introduction

In this Jupyter Notebook, I plan to implement the same data preprocessing and analysis techniques done in the paper "Human stereoEEG recordings reveal network dynamics of decision-making in a rule-switching task" by Marije ter Wal et al.

Details of data preprocessing, artifact rejection, wavelet analysis, and classfiers can be found in the "Methodology" section of the paper.

## Imports

In [70]:
import h5py 
import mat73
import numpy as np
import mne
from matplotlib import pyplot as plt
from scipy import signal, stats

## Data Paths

In [2]:
# File paths for raw and setup files for subject #06
setup_path = '/run/user/1000/gvfs/smb-share:server=10.162.37.21,share=main/Public/EFRI/1_formatted/SUBJECT06/EFRI06_WAR_SES1_Setup.mat'
raw_path = '/run/user/1000/gvfs/smb-share:server=10.162.37.21,share=main/Public/EFRI/1_formatted/SUBJECT06/EFRI06_WAR_SES1_Raw.mat'

## Load raw data using h5py library

In [3]:
# Load the raw matlab file, EFRI06_WAR_SES1_Raw.mat
raw_file = h5py.File(raw_path)

In [4]:
raw_file

<HDF5 file "EFRI06_WAR_SES1_Raw.mat" (mode r)>

In [5]:
# Access and print the sampling frequency 
Fs = raw_file['Fs'][0][0]
print(Fs)

2000.0


In [6]:
# Load in EEG data
lfp_data = raw_file['lfpdata']

In [7]:
lfp_data

<HDF5 dataset "lfpdata": shape (133, 3934000), type "<f8">

In [74]:
# Get a subset of time elements from the first channel
ch0 = lfp_data[0,:10000]

In [9]:
# Access the setup data
setup_data = mat73.loadmat(setup_path)

In [10]:
setup_data.keys()

dict_keys(['elec_area', 'elec_ind', 'elec_name', 'filters', 'trial_times', 'trial_words'])

## Create MNE data objects

In [11]:
# Access the names of the EEG channels
ch_names = setup_data['elec_name']

In [13]:
# Get the name for the first channel
ch_names[0]

"B'10"

In [75]:
# Create an MNE Info data object
info = mne.create_info([ch_names[0]], 2000.0)

In [76]:
# Create an MNE Raw Array object
raw = mne.io.RawArray(ch0.reshape(1,-1), info)

Creating RawArray with float64 data, n_channels=1, n_times=10000
    Range : 0 ... 9999 =      0.000 ...     5.000 secs
Ready.


##  Preprocessing


In [124]:
# Filter Data Bandpass 1.5-300 Hz 
filt = raw.copy().filter(1.5,300,'all',method="iir")

# notch filter 60 Hz harmonics
for notchfreq in [60,120,180,240,300]:
    filt = filt.copy().notch_filter(notchfreq, 'all', method="iir")

# decimate to 600 Hz 
decFactor = int(Fs/600)
filt = filt.get_data()[:,::decFactor]

No data channels found. The highpass and lowpass values in the measurement info will not be updated.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1.5 - 3e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.50, 300.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

IIR filter parameters
---------------------
Butterworth bandstop zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 59.35, 60.65 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 1.2e+02 - 1.2e+02 Hz

IIR filter parameters
---------------------
Butterworth bandstop zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forwa

In [125]:
filt

array([[-9.39438713e-03, -1.50138990e+01, -2.77560977e+01, ...,
         1.10905957e+01,  2.77916708e+01, -2.52458520e-01]])

In [127]:
# Wavelet covers frequencies between 5 and 152 Hz at 50 (semi)logarithmic intervals
wavelet_freqs = np.logspace(np.log2(5),np.log2(152),num=50,base=2)
lap_ref_data = np.zeros(filt.shape)

In [128]:
# Complex Morlet mother wavelet of 4 cycles wide
#tfr_by_chan_list = []
tfr_array = np.zeros((lap_ref_data.shape[0], len(wavelet_freqs), int(lap_ref_data.shape[1])))
for ch in range(lap_ref_data.shape[0]):
    tfr = mne.time_frequency.tfr_array_morlet(lap_ref_data[ch,:].reshape(1,1,-1),600,wavelet_freqs,n_cycles=4,output='power',n_jobs=1,)
    #downsample = np.convolve(tfr[0,0,0,:], np.ones(50, ), mode='valid')[::25]/50. #Fs = 500; 1/Fs = .002 s; .1 sec/.002 sec = 50 samples for 100 ms. hence 50. then downsample every 25 because going from 500 Hz to 20 Hz final time resolution
    downsample = signal.convolve2d(tfr[0,0,:,:], (1.0/50)* np.ones((1,50)), mode='same')[:,:]
    #tfr_by_chan_list.append(downsample)
    downsample = np.log(downsample) # natural log normalization to make frequencies more comparable
    downsample = stats.zscore(downsample,axis=1)
    tfr_array[ch,:,:] = downsample
    print(ch)

0


/tmp/ipykernel_26287/1091223578.py:9: RuntimeWarning: divide by zero encountered in log
  downsample = np.log(downsample) # natural log normalization to make frequencies more comparable
